<a href="https://colab.research.google.com/github/wgcv/03MAIR-Algoritmos-de-Optimizacion/blob/main/Trabajo-Final/Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Washington Gustavo Cevallos Vaca  <br>
Url: https://github.com/wgcv/03MAIR-Algoritmos-de-Optimizacion/blob/main/Trabajo-Final/Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb<br>
Google Colab: https://colab.research.google.com/github/wgcv/03MAIR-Algoritmos-de-Optimizacion/blob/main/Trabajo-Final/Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:

## Problema 3. Generación de Tribunales(I)
- Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean
presentar su Trabajo Fin de Máster (TFM).
- Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de los
siguientes roles: Presidente, Secretario o Vocal.
- Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana del 15 al 19 de abril:

  - Número de profesores : 10
  - Número de tribunales : 15


- 1 indica que profesor tiene disponibilidad
- 0 en caso contrario

- Hay 15 alumnos, por lo que se deben configurar 15 tribunales buscando la configuración
más equilibrada posible en cuanto a la cantidad de tribunales asignados a cada profesor, es
decir, evitando que un profesor tenga muchos tribunales y otros pocos.
- Obviamente ningún profesor puede asistir a dos tribunales a la misma fecha/hora y no puede ser convocado a un tribunal al que no tiene disponibilidad.

- Número de profesores : 10
- Número de tribunales : 15
- Disponibilidad/Roles : https://bit.ly/41QWk8o




                                        

#Modelo

# ¿Como represento el espacio de soluciones?
El espacio de soluciones lo podemos pensar como todas las formas posibles de armar los tribunales. Si bien el problema no busca equilibrar los roles, igual hay que considerar las distintas permutaciones que pueden tener los profesores. O sea, un tribunal formado por los mismos profes pero con roles distintos va a dar el mismo costo, pero no es la misma solución, porque lo que nos importa es saber qué rol le toca a cada uno.

### ¿Cual es la función objetivo?
La función objetivo busca **maximizar el valor negativo de la varianza de la carga de trabajo entre los profesores**, penalizando además el incumplimiento de restricciones.

Sea $X = {x_1, x_2, \dots, x_n}$ el conjunto de tribunales asignados a cada profesor. La función de aptitud se define como:

$$
f(X) = -\mathrm{Var}(X) - \text{Costo_Total}
$$

donde:

* $\mathrm{Var}(X)$ es la varianza de la carga de trabajo entre profesores.
* $\text{Costo_Total}$ es la suma de penalizaciones por restricciones incumplidas.

El algoritmo busca acercar $f(X)$ lo más posible a $0$, lo que indicaría que **no hay penalizaciones** y que la **distribución de tribunales entre profesores es uniforme**.


### ¿Como implemento las restricciones?

Los tribunales que no cumplan con las restricciones van a recibir una puntuación negativa, mientras que los que sí las cumplan van a recibir puntuaciones más altas. Además, se va a premiar aquellas configuraciones donde la distribución de horarios entre los profesores sea lo más uniforme posible, o sea, que todos tengan más o menos la misma cantidad de tribunales.

**Teniendo en cuenta que:**

&emsp;$X_{i,j} = 1$ indica que el profesor $i$ asiste al tribunal $j$, y $0$ lo contrario<br/>
&emsp;$T_{j}$ es el conjunto de roles asignados al tribunal $j$<br/>
&emsp;$A_{i,j} = 1$ indica que el profesor $i$ está disponible en el horario $j$, y $0$ lo contrario<br/>
&emsp;$Q_i$ es el conjunto de roles del profesor $i$, donde las opciones pueden ser Presidente, Secretario o Vocal<br/>



**Las restricciones del problema son:**

&emsp;Un profesor no puede asignarse si no está disponible: $X_{i,j}=1 \Rightarrow A_{i,j}=1$<br/>
&emsp;Un profesor no puede asignarse dos veces al mismo tribunal<br/>
&emsp;Un tribunal no puede tener más de 3 miembros: $|T_j| \leq 3$<br/>
&emsp;El profesor solo puede asignarse si su rol no está ya cubierto en el tribunal: $Q_i \cap T_j = \emptyset$<br/>
&emsp;El profesor debe tener al menos un rol posible: $1 \leq |Q_i| \leq 3$<br/>

#Análisis

#### ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

La complejidad del algoritmo se puede analizar por partes:  

- **generar_poblacion()**: depende del tamaño de la población y de los tribunales, profesores y horas, así que `O(tamaño)` si consideramos que lo demás es constante.  
- **fitness()**: recorre profesores y horas, `O(num_profesores * num_horas)`.  
- **seleccion_ruleta()**: realiza búsquedas binarias, `O(tamaño * log(tamaño))`.  
- **cruce() y mutacion()**: son lineales respecto a sus variables, con operaciones simples `O(horas+bloques_tiempo)` y `O(horas×profesores)` respectivamente.  


En cuanto al problema en sí, crece muy rápido: asignar roles a un tribunal tiene $720$ combinaciones, y con 15 tribunales y distintas horas, serían $720^{15}$ combinaciones posibles.

Si además consideramos la elección de horas:  

$$
\binom{35}{15} \cdot 720^{15}
$$

Por eso, la complejidad global del problema es exponencial:  

$$
O(n^k)
$$

Con un espacio de soluciones muy grande.

#Diseño


### ¿Que técnica utilizo? ¿Por qué?

Para resolver este problema se implementa un algoritmo genético. La idea es generar un conjunto de soluciones iniciales e ir iterando sobre ellas mediante selección, cruce y mutación hasta que las soluciones mejoren. Debido al tamaño del espacio de soluciones, opciones como fuerza bruta o algoritmos deterministas quedan completamente descartadas, ya que se vuelven muy costosas a medida que crece la dimensión del problema.
Los algoritmos genéticos son heurísticos, así que no vamos encontrar la solución óptima, al incorporar mecanismos aleatorios, pueden dar resultados distintos para los mismos datos de entrada pero la solución va a ser suficientemente buena para el tipo de problema.

#### La implementación sigue estos pasos:
1.   Población inicial: Se genera aleatoriamente. Cada individuo es una matriz de 10x35 donde los valores indican el rol del profesor en cada tribunal: 0 si no participa, 1 si es Presidente, 2 si es Secretario y 3 si es Vocal.
2.   Función de fitness: Primero verifica las restricciones de disponibilidad, roles y número de tribunales. Si alguna no se cumple, se aplica una penalización. Si se cumplen, se devuelve la varianza negativa sobre el número de tribunales por profesor. Los valores más cercanos a 0 son los mejores; cuanto más negativos, peor es la solución.
3.   Selección: Se usa el mecanismo de ruleta.
4.   Cruce: Se elige aleatoriamente un punto de corte por columna y se recombinan los padres respetando las restricciones para generar 2 hijos.
5.   Mutación: Se define un ratio de mutación y, si un gen muta, su valor cambia aleatoriamente entre los roles disponibles.
6.   Elitismo: Los mejores individuos de cada generación pasan directamente a la siguiente, asegurando que las mejores soluciones no se pierdan.

Al finalizar las generaciones definidas, se muestra el mejor individuo encontrado.

## Solución en código

### Librerías

In [ ]:
import numpy as np
import random

### Representación de los datos

In [ ]:
# Basado en el excel, se genera la siguiente matriz 10x35
disponibilidad = np.array([
  [0,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0],
  [1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
  [0,0,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,0,1,0,1],
  [1,0,1,0,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0],
  [1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,0],
  [1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,0,1],
  [0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,0,1],
  [1,1,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0],
  [1,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1],
  [1,1,0,1,1,0,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1]
])
# [Presidente ,Secretario, Vocal]
# Donde 1 es el rol y 0 no tiene dicho rol
roles = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
    [0, 1, 1],
    [0, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
])

n_paneles = 15

nombres_profesores =  {
        0: 'RRD',
        1: 'QYV',
        2: 'LHL',
        3: 'HLC',
        4: 'MSB',
        5: 'PMQ',
        6: 'QWF',
        7: 'EBB',
        8: 'IOE',
        9: 'IOA',
    }

### Generar población inicial

In [ ]:
def generar_poblacion(tam_poblacion, disponibilidad, matriz_roles, total_tribunales):
    poblacion = []
    n_docentes, n_franjas = disponibilidad.shape

    for _ in range(tam_poblacion):
        asignacion = np.zeros((n_docentes, n_franjas), dtype=int)
        tribunales_completados = 0

        while tribunales_completados < total_tribunales:
            # Buscar franjas horarias con espacio disponible
            franjas_libres = [
                franja for franja in range(n_franjas)
                if np.count_nonzero(asignacion[:, franja]) < 3
            ]

            if not franjas_libres:
                break

            franja_elegida = np.random.choice(franjas_libres)
            roles_cubiertos = 0

            # Codificación de roles: 1 = presidente, 2 = secretario, 3 = vocal
            for id_rol in range(1, 4):
                candidatos = np.where(
                    (disponibilidad[:, franja_elegida] == 1) &
                    (matriz_roles[:, id_rol - 1] == 1)
                )[0]

                # Excluir docentes ya asignados a esa franja
                candidatos = [d for d in candidatos if asignacion[d, franja_elegida] == 0]

                if candidatos:
                    elegido = np.random.choice(candidatos)
                    asignacion[elegido, franja_elegida] = id_rol
                    roles_cubiertos += 1

            if roles_cubiertos == 3:
                tribunales_completados += 1

        poblacion.append(asignacion)

    return poblacion

In [ ]:
def evaluar_individuo(asignacion, disponibilidad, matriz_roles, paneles_requeridos):
    COSTO = 100
    costo_total = 0
    n_docentes, n_franjas = asignacion.shape

    # Penalizar disponibilidad y roles inválidos
    docentes_activos = np.argwhere(asignacion > 0)
    for docente, franja in docentes_activos:
        if disponibilidad[docente, franja] == 0:
            costo_total += COSTO
        id_rol = asignacion[docente, franja]
        if matriz_roles[docente, id_rol - 1] == 0:
            costo_total += COSTO

    # Penalizar franjas con composición inválida
    for franja in range(n_franjas):
        columna = asignacion[:, franja]
        if np.count_nonzero(columna) > 3:
            costo_total += COSTO
        for id_rol in range(1, 4):
            if np.sum(columna == id_rol) > 1:
                costo_total += COSTO

    # Verificar número exacto de paneles completos
    paneles_validos = sum(
        1 for franja in range(n_franjas)
        if all(np.sum(asignacion[:, franja] == rol) == 1 for rol in range(1, 4))
    )
    if paneles_validos != paneles_requeridos:
        costo_total += COSTO

    # Penalizar distribución desigual de carga entre docentes
    carga_por_docente = np.count_nonzero(asignacion, axis=1)
    varianza_carga = np.var(carga_por_docente)

    return -(varianza_carga*10) - costo_total

In [ ]:
def ruleta_estocastica(generacion, puntajes):
    # Ajustar puntajes para que todos sean positivos
    base = min(puntajes)
    puntajes_ajustados = [1 + (p - base) for p in puntajes]

    # Calcular probabilidades relativas
    total = sum(puntajes_ajustados)
    probabilidades = [p / total for p in puntajes_ajustados]

    # Distribución acumulada para búsqueda binaria
    distribucion = np.cumsum(probabilidades)

    # Seleccionar individuos por sorteo proporcional
    seleccionados = [
        generacion[np.searchsorted(distribucion, np.random.rand())]
        for _ in range(len(generacion))
    ]

    return seleccionados

In [ ]:
def cruzar_progenitores(alpha, beta, disponibilidad, matriz_roles):
    descendiente_1 = np.copy(alpha)
    descendiente_2 = np.copy(beta)
    _, n_franjas = disponibilidad.shape

    # Franjas donde ambos progenitores tienen paneles completos
    franjas_validas = [
        franja for franja in range(n_franjas)
        if np.count_nonzero(alpha[:, franja]) == 3 and
           np.count_nonzero(beta[:, franja]) == 3
    ]

    # Intercambiar columnas en franjas válidas
    for franja in franjas_validas:
        descendiente_1[:, franja], descendiente_2[:, franja] = (
            beta[:, franja].copy(),
            alpha[:, franja].copy()
        )

    return descendiente_1, descendiente_2

In [ ]:
def mutar_asignacion(cromosoma, disponibilidad, matriz_roles):
    _, n_franjas = disponibilidad.shape

    for franja in range(n_franjas):
        # Escoger un rol al azar para intentar reasignar
        rol_objetivo = np.random.choice([1, 2, 3])

        portadores = np.where(cromosoma[:, franja] == rol_objetivo)[0]
        if portadores.size == 0:
            continue

        # Buscar sustitutos válidos que estén libres en esa franja
        sustitutos = np.where(
            (disponibilidad[:, franja] == 1) &
            (matriz_roles[:, rol_objetivo - 1] == 1) &
            (cromosoma[:, franja] == 0)
        )[0]

        if sustitutos.size > 0:
            saliente = np.random.choice(portadores)
            entrante = np.random.choice(sustitutos)
            cromosoma[saliente, franja] = 0
            cromosoma[entrante, franja] = rol_objetivo

    return cromosoma

In [ ]:
tam_poblacion = 100
n_generaciones = 200
prob_mutacion = 0.3
prob_cruce = 0.8 # La academia recomienda entre 0.7 y 0.9
n_elites = int(tam_poblacion * 0.05)
ciclos_sin_mejora = 0
mejor_historico = float('-inf')

# Crear la generacion base
generacion_base = generar_poblacion(tam_poblacion, disponibilidad, roles, n_paneles)

# Evaluar la generacion base
puntajes = [evaluar_individuo(cromosoma, disponibilidad, roles, n_paneles) for cromosoma in generacion_base]

# Bucle principal del algoritmo genetico
for ciclo in range(n_generaciones):
    # Seleccion proporcional al puntaje
    candidatos = ruleta_estocastica(generacion_base, puntajes)

    # Preservar los mejores cromosomas (elitismo)
    idx_elites = np.argsort(puntajes)[-n_elites:]
    siguiente = [generacion_base[i] for i in idx_elites]

    # Contar ciclos
    if max(puntajes) > mejor_historico:
        mejor_historico = max(puntajes)
        ciclos_sin_mejora = 0
    else:
        ciclos_sin_mejora += 1

    # Si lleva 20 ciclos sin mejorar, reiniciar parte de la población
    if ciclos_sin_mejora >= 20:
        nuevos = generar_poblacion(tam_poblacion // 2, disponibilidad, roles, n_paneles)
        generacion_base[n_elites:n_elites + len(nuevos)] = nuevos
        ciclos_sin_mejora = 0

    # Construir la siguiente generacion
    while len(siguiente) < tam_poblacion:
        progenitor_a, progenitor_b = random.sample(candidatos, 2)

        # Cruce probabilistico
        if np.random.rand() < prob_cruce:
            desc_1, desc_2 = cruzar_progenitores(progenitor_a, progenitor_b, disponibilidad, roles)
        else:
            desc_1, desc_2 = progenitor_a.copy(), progenitor_b.copy()

        # Mutacion probabilistica
        if np.random.rand() < prob_mutacion:
            desc_1 = mutar_asignacion(desc_1, disponibilidad, roles)
            desc_2 = mutar_asignacion(desc_2, disponibilidad, roles)

        siguiente.append(desc_1)
        if len(siguiente) < tam_poblacion:
            siguiente.append(desc_2)

    # Actualizar poblacion y puntajes
    generacion_base = siguiente
    puntajes = [evaluar_individuo(cromosoma, disponibilidad, roles, n_paneles) for cromosoma in generacion_base]

    print(f"Ciclo {ciclo}: Mejor puntaje = {max(puntajes)}")

# Resultado final
mejor_puntaje = max(puntajes)
mejor_cromosoma = generacion_base[puntajes.index(mejor_puntaje)]

print("Mejor cromosoma encontrado:")
print(mejor_cromosoma)
print(f"Puntaje final: {mejor_puntaje}")

Ciclo 0: Mejor puntaje = -8.5
Ciclo 1: Mejor puntaje = -8.5
Ciclo 2: Mejor puntaje = -6.5
Ciclo 3: Mejor puntaje = -6.5
Ciclo 4: Mejor puntaje = -6.5
Ciclo 5: Mejor puntaje = -4.5
Ciclo 6: Mejor puntaje = -4.5
Ciclo 7: Mejor puntaje = -4.5
Ciclo 8: Mejor puntaje = -4.5
Ciclo 9: Mejor puntaje = -4.5
Ciclo 10: Mejor puntaje = -4.5
Ciclo 11: Mejor puntaje = -4.5
Ciclo 12: Mejor puntaje = -2.5
Ciclo 13: Mejor puntaje = -2.5
Ciclo 14: Mejor puntaje = -2.5
Ciclo 15: Mejor puntaje = -2.5
Ciclo 16: Mejor puntaje = -2.5
Ciclo 17: Mejor puntaje = -2.5
Ciclo 18: Mejor puntaje = -2.5
Ciclo 19: Mejor puntaje = -2.5
Ciclo 20: Mejor puntaje = -2.5
Ciclo 21: Mejor puntaje = -2.5
Ciclo 22: Mejor puntaje = -2.5
Ciclo 23: Mejor puntaje = -2.5
Ciclo 24: Mejor puntaje = -2.5
Ciclo 25: Mejor puntaje = -2.5
Ciclo 26: Mejor puntaje = -2.5
Ciclo 27: Mejor puntaje = -2.5
Ciclo 28: Mejor puntaje = -2.5
Ciclo 29: Mejor puntaje = -2.5
Ciclo 30: Mejor puntaje = -2.5
Ciclo 31: Mejor puntaje = -2.5
Ciclo 32: Mejor pu

In [ ]:
def mostrar_resultado(cromosoma, nombres_profesores):
    n_docentes, n_franjas = cromosoma.shape
    num_panel = 1

    print("===== DISTRIBUCIÓN DE TRIBUNALES =====")
    for franja in range(n_franjas):
        columna = cromosoma[:, franja]

        # Verificar si la franja tiene un panel completo
        if all(np.sum(columna == rol) == 1 for rol in range(1, 4)):
            presidente = nombres_profesores[np.where(columna == 1)[0][0]]
            secretario = nombres_profesores[np.where(columna == 2)[0][0]]
            vocal      = nombres_profesores[np.where(columna == 3)[0][0]]

            print(f"""
  Panel {num_panel}
  Franja {franja + 1}
  Presidente: {presidente}
  Secretario: {secretario}
  Vocal: {vocal}""")
            num_panel += 1

    print("\n\n======================================\n\n")
    print("Carga por docente:")
    for idx in range(n_docentes):
        turnos = np.count_nonzero(cromosoma[idx, :])
        print(f"  {nombres_profesores[idx]}: {turnos} turno(s)")

# Mostrar el mejor resultado
mostrar_resultado(mejor_cromosoma, nombres_profesores)

===== DISTRIBUCIÓN DE TRIBUNALES =====

  Panel 1
  Franja 1
  Presidente: MSB
  Secretario: QYV
  Vocal: EBB

  Panel 2
  Franja 3
  Presidente: IOE
  Secretario: QWF
  Vocal: HLC

  Panel 3
  Franja 4
  Presidente: RRD
  Secretario: EBB
  Vocal: PMQ

  Panel 4
  Franja 8
  Presidente: RRD
  Secretario: EBB
  Vocal: MSB

  Panel 5
  Franja 10
  Presidente: LHL
  Secretario: IOE
  Vocal: RRD

  Panel 6
  Franja 14
  Presidente: IOA
  Secretario: HLC
  Vocal: LHL

  Panel 7
  Franja 15
  Presidente: QYV
  Secretario: MSB
  Vocal: EBB

  Panel 8
  Franja 18
  Presidente: LHL
  Secretario: QWF
  Vocal: HLC

  Panel 9
  Franja 20
  Presidente: IOA
  Secretario: QYV
  Vocal: EBB

  Panel 10
  Franja 22
  Presidente: IOA
  Secretario: IOE
  Vocal: PMQ

  Panel 11
  Franja 27
  Presidente: PMQ
  Secretario: RRD
  Vocal: IOA

  Panel 12
  Franja 28
  Presidente: MSB
  Secretario: HLC
  Vocal: QYV

  Panel 13
  Franja 29
  Presidente: PMQ
  Secretario: IOA
  Vocal: QWF

  Panel 14
  Franja 33
 